In [2]:
import numpy as np
import pandas as pd
import requests
import os
import json

In [3]:
congressapi = os.environ['congressapi']
opensecretsapi = os.environ['opensecretsapi']

# Congress Bills API for the 118th Congress

In [4]:
root = 'https://api.congress.gov/v3'
congress = '118'
endpoint = f'/bill/{congress}'

In [5]:
parameters = {'format': 'json',
             'offset': 0,
              'limit': 250,
              'sort': 'updateDate+desc',
              'api_key': congressapi}

In [6]:
r = requests.get('http://httpbin.org/user-agent')
useragent = json.loads(r.text)['user-agent']

In [7]:
headers = {'User-Agent': useragent,
          'From': 'pdy2bw@virginia.edu'}

In [8]:
r = requests.get(root + endpoint,
                params = parameters,
                headers = headers)

In [9]:
r

<Response [200]>

In [10]:
myjson = json.loads(r.text)

In [11]:
myjson['bills'][0]['number']

'4367'

In [12]:
#[x['number'] for x in myjson['bills']]

In [13]:
with open('bills.json', 'w') as f:
    json.dump(myjson, f)

In [14]:
#can use jsonhero.io to see things easier with non-private info

In [15]:
billsdf = pd.json_normalize(myjson,
                           record_path = ['bills'])

In [16]:
root = 'http://www.opensecrets.org/api/'
parameters = {'method' : 'getLegislators',
             'apikey' : opensecretsapi,
             'id' : 'VA',
             'output' : 'json'}

In [17]:
r = requests.get(root, params=parameters, headers=headers)

In [18]:
r

<Response [200]>

In [25]:
#thursday sept 28 - house bills
root = 'https://api.congress.gov/v3'
congress = '118'
bill_type = "hr"
endpoint = f'/bill/{congress}/{bill_type}'
r = requests.get('http://httpbin.org/user-agent')
useragent = json.loads(r.text)['user-agent']
headers = {'User-Agent': useragent,
          'From': 'pdy2bw@virginia.edu'}

house_bills = []
for i in range(0, 1001, 250):
    parameters = {'format': 'json',
                    'offset': i,
                  'limit': 250,
                  'sort': 'updateDate+desc',
                  'api_key': congressapi
    }
    r = requests.get(root + endpoint, params = parameters, headers = headers)
    myjson = json.loads(r.text)
    bills = myjson['bills']
    house_bills.append(bills)

In [26]:
df1 = pd.DataFrame.from_dict(house_bills[0])
df2 = pd.DataFrame.from_dict(house_bills[1])
df3 = pd.DataFrame.from_dict(house_bills[2])
df4 = pd.DataFrame.from_dict(house_bills[3])
df5 = pd.DataFrame.from_dict(house_bills[4])
frames = [df1, df2, df3, df4, df5]
house_bills = pd.concat(frames, ignore_index = True)

In [ ]:
#thursday sept 28 - senate bills
root = 'https://api.congress.gov/v3'
congress = '118'
bill_type = "s"
endpoint = f'/bill/{congress}/{bill_type}'
r = requests.get('http://httpbin.org/user-agent')
useragent = json.loads(r.text)['user-agent']
headers = {'User-Agent': useragent,
          'From': 'pdy2bw@virginia.edu'}

senate_bills = []
for i in range(0, 1001, 250):
    parameters = {'format': 'json',
                    'offset': i,
                  'limit': 250,
                  'sort': 'updateDate+desc',
                  'api_key': congressapi
    }
    r = requests.get(root + endpoint, params = parameters, headers = headers)
    myjson = json.loads(r.text)
    bills = myjson['bills']
    senate_bills.append(bills)

df1 = pd.DataFrame.from_dict(senate_bills[0])
df2 = pd.DataFrame.from_dict(senate_bills[1])
df3 = pd.DataFrame.from_dict(senate_bills[2])
df4 = pd.DataFrame.from_dict(senate_bills[3])
df5 = pd.DataFrame.from_dict(senate_bills[4])
frames = [df1, df2, df3, df4, df5]
senate_bills = pd.concat(frames, ignore_index = True)

In [ ]:
#thursday sept 28 - sponsors
root = 'https://api.congress.gov/v3'
congress = '118'
bill_type = "s"
cosponsors = []

for i in range(0, 100):
    bill_number = senate_bills.iloc[i]['number']
    endpoint = f'/bill/{congress}/{bill_type}/{bill_number}/cosponsors'
    parameters = {'format': 'json',
                  'sort': 'updateDate+desc',
                  'api_key': congressapi
    }
    r = requests.get(root + endpoint, params = parameters, headers = headers)
    myjson = json.loads(r.text)
    cosponsor = myjson['cosponsors']
    cosponsors.append({"Bill Number": f"S {bill_number}", "Cosponsors": cosponsor})

cosponsors = pd.DataFrame.from_dict(cosponsors)
cosponsors   

In [44]:
root = 'http://www.opensecrets.org/api/'
parameters = {'method' : 'getLegislators',
             'apikey' : opensecretsapi,
             'id' : 'VA',
             'output' : 'json'}
headers = {'User-Agent': useragent,
          'From': 'pdy2bw@virginia.edu'}
r = requests.get(root, params=parameters, headers=headers)
myjson = json.loads(r.text)

cids = []
for i in range(0, len(myjson['response']['legislator'])):
    cid = myjson['response']['legislator'][i]['@attributes']['cid']
    cids.append(cid)
cids

['N00029459',
 'N00048315',
 'N00002147',
 'N00051961',
 'N00045557',
 'N00042296',
 'N00041418',
 'N00036018',
 'N00032029',
 'N00041002',
 'N00029891',
 'N00033177',
 'N00002097']

In [49]:
financial_information = []
for i in range(0, len(cids)):
    parameters = {'method' : 'memPFDprofile',
             'apikey' : opensecretsapi,
             'cid' : cids[i],
             'output' : 'json'}
    r = requests.get(root, params=parameters, headers=headers)
    myjson = json.loads(r.text)
    data = myjson['response']['member_profile']
    financial_information.append({'CID': cids[i], "Financial Information": data})
    
financial_information = pd.DataFrame.from_dict(financial_information)
financial_information
    

,CID,Financial Information
0,N00029459,"{'@attributes': {'name': 'Wittman, Rob', 'data..."
1,N00048315,"{'@attributes': {'name': '', 'data_year': '201..."
2,N00002147,"{'@attributes': {'name': 'Scott, Bobby', 'data..."
3,N00051961,"{'@attributes': {'name': '', 'data_year': '201..."
4,N00045557,"{'@attributes': {'name': '', 'data_year': '201..."
5,N00042296,"{'@attributes': {'name': '', 'data_year': '201..."
6,N00041418,"{'@attributes': {'name': '', 'data_year': '201..."
7,N00036018,"{'@attributes': {'name': '', 'data_year': '201..."
8,N00032029,"{'@attributes': {'name': 'Griffith, Morgan', '..."
9,N00041002,"{'@attributes': {'name': '', 'data_year': '201..."
